In [2]:
import random
import pandas as pd

# Kano Analysis
## Kano analysis is a framework for prioritizing product features from a backlog
## To conduct a Kano analysis, we need:
- A set of users to survey
- A small set of features to study
- Two questions about each feature:
    - Functional question: How do you feel when Feature X is available?
    - Dysfunctional question: How do you feel when Feature X is unavailable?
- A limited list of valid answers to each question

In [3]:
# simulate 100 users to be surveyed
users = [f'User {i}' for i in range(1, 101)]

# simulate 5 features to be studied
features = [f'Feature {i}' for i in range(1, 6)]

# specify the valid answers for each question
potentialAnswers = ['Like it', 'Expect it', 'Don\'t care', 'Can live with it', 'Dislike it']

In [4]:
# define a weight matrix that approximates the relationship between functional and dysfunctional answers about the same feature
weights = pd.DataFrame(columns = potentialAnswers, index = potentialAnswers)
weights.loc['Like it'] = [0.05, 0.05, 0.2, 0.55, 0.15]
weights.loc['Expect it'] = [0.01, 0.01, 0.01, 0.85, 0.12]
weights.loc['Don\'t care'] = [0.01, 0.15, 0.7, 0.1, 0.13]
weights.loc['Can live with it'] = [0.59, 0.39, 0.01, 0.005, 0.005]
weights.loc['Dislike it'] = [0.85, 0.01, 0.01, 0.12, 0.01]
weights

,Like it,Expect it,Don't care,Can live with it,Dislike it
Like it,0.05,0.05,0.2,0.55,0.15
Expect it,0.01,0.01,0.01,0.85,0.12
Don't care,0.01,0.15,0.7,0.1,0.13
Can live with it,0.59,0.39,0.01,0.005,0.005
Dislike it,0.85,0.01,0.01,0.12,0.01


In [5]:
weights.sum(axis = 1)

Like it             1.00
Expect it           1.00
Don't care          1.09
Can live with it    1.00
Dislike it          1.00
dtype: float64

In [7]:
# simulate a set of answers for each user
answers = pd.DataFrame()
for user in users:
    for feature in features:
        # simulate variation in answer to functional question
        functionalAnswer = random.choices(potentialAnswers, weights = [0.15, 0.60, 0.10, 0.05, 0.10], k = 1)[0]
        
        # expect an inverse response to the dysfunctional question - with some variation
        dysfunctionalAnswer = random.choices(potentialAnswers, weights = weights.loc[functionalAnswer], k = 1)[0]
        
        # combine these elements
        answer = pd.DataFrame([[user, feature, functionalAnswer, dysfunctionalAnswer]]
                              , columns = ['User', 'Feature', 'Functional Answer', 'Dysfunctional Answer'])
        
        # stack responses
        answers = pd.concat([answers, answer], ignore_index = True)
        
# convert answers to ordered categorical features for ease of evaluation
answers['Functional Answer'] = pd.Categorical(answers['Functional Answer']
                                              , categories = potentialAnswers, ordered = True)
answers['Dysfunctional Answer'] = pd.Categorical(answers['Dysfunctional Answer']
                                                 , categories = potentialAnswers, ordered = True)

In [12]:
# verify that answers are sensible
pd.pivot_table(answers, index = 'Functional Answer', columns = 'Dysfunctional Answer', aggfunc = 'count', values = 'User')

Dysfunctional Answer,Like it,Expect it,Don't care,Can live with it,Dislike it
Functional Answer,,,,,
Like it,4,2,16,44,9
Expect it,3,3,5,268,35
Don't care,2,9,30,5,3
Can live with it,14,7,0,0,0
Dislike it,34,0,0,7,0


In [9]:
answers.to_csv('./data/Survey Data.csv', index = False)